In [1]:
from pyspark import SparkConf, SparkContext, HiveContext
import re
conf = (SparkConf()
        .set("spark.executor.instances", 16)
        .set("spark.driver.maxResultSize", "16g")
        .set('spark.driver.memory','16g')
        .set("spark.executor.memory", '4g')
        .set("spark.yarn.executor.memoryOverhead", 1048)
       )

try:
    sc.stop()
except:
    pass
sc = SparkContext()
hc = HiveContext(sc)

In [2]:

def metrics(y_true,y_score,lift = None, return_str = False):
    import sklearn
    import collections
    
    if True:
        
        res = collections.OrderedDict()
        samp_size = len(y_true)
        res['Sample size'] = samp_size
        res['Posit share'] = sum(y_true) * 1./ samp_size
        res['Sample size'] = len(y_true)
        res['AUC ROC'] = sklearn.metrics.roc_auc_score(y_true = y_true, y_score = y_score)
        res['AUC PR'] = sklearn.metrics.average_precision_score( y_true,  y_score)
        res['Log loss'] = sklearn.metrics.log_loss(y_true = y_true, y_pred = y_score)
        if lift:
            predictions_and_labels = sorted(zip(y_score,y_true), key = lambda e:-e[0])
            for l in lift:
                res['Lift ' + str(l)] = sum([e[1] for e in predictions_and_labels[:int(l * samp_size)]]) * 1. / int(l * samp_size) / res['Posit share']                
        if return_str:
            res = '\n'.join(['{:<12}: {:.5f}'.format(k,v) for (k,v) in res.items()]) + '.'
        return res

In [16]:
query1 = '''


create table user_kposminin.urlfr_scores_threshold_tst as
  select 
    v.id, 
    v.load_src, 
    max(m.phone_num) as phone_num, 
    max(if(u.id is Null,0,1)) as label, 
    max(t1.coeff) as score1, 
    max(t2.coeff) as score2, 
    max(t3.coeff) as score3 
  from 
   
   (select id, load_src, url_fragment from prod_odd.visit_feature where ymd = '2017-06-13') v
   left join     (select 
       uid_str as id,
       property_value as phone_num,
       load_src
     from
       prod_dds.md_uid_property 
     where
       property_cd = 'PHONE'
    ) m on v.id = m.id and v.load_src = m.load_src
   left join 
    (
      select id
      from prod_features_liveinternet.user_action 
      where ymd between '2017-06-13' and date_add('2017-06-13',3)
      and action_type = 'tinkoff_platinum_approved_application'
    ) u on u.id = v.id
   left join dds_dic. max_coeff_url_fragment_score t1 on t1.url_fragment = v.url_fragment and t1.segment_nm = 'la_apppr_ccall_2_1'
   left join dds_dic. max_coeff_url_fragment_score t2 on t2.url_fragment = v.url_fragment and t2.segment_nm = 'la_apppr_ccall_2'
   left join dds_dic. max_coeff_url_fragment_score t3 on t3.url_fragment = v.url_fragment and t3.segment_nm = 'la_stand_ccall_2'

   group by
    v.id, v.load_src 

'''


query2 = '''

'''

In [3]:
data = hc.sql('''
   select * from user_kposminin.urlfr_scores_threshold_test 
   '''
   ).toPandas()

[0.5,
 0.3,
 0.2,
 0.1,
 0.05,
 0.03,
 0.02,
 0.01,
 0.004,
 0.0036,
 0.0032,
 0.0028,
 0.0024,
 0.002,
 0.0016,
 0.0012,
 0.0008,
 0.0004]

In [5]:
import sklearn,sklearn.metrics
print(metrics(y_true = data['approve_label'].fillna(0),y_score = data['score1'].fillna(-100),
              lift = [0.5,0.3,0.2,0.1,0.05,0.03,0.02,0.01] + [10.**4*i /(25 * 10**6) for i in range(10,0,-1)], return_str = True))

Sample size : 25031520.00000
Posit share : 0.00005
AUC ROC     : 0.74133
AUC PR      : 0.00808
Log loss    : 0.00166
Lift 0.5    : 1.58271
Lift 0.3    : 2.14464
Lift 0.2    : 2.66833
Lift 0.1    : 4.13965
Lift 0.05   : 6.71654
Lift 0.03   : 8.45110
Lift 0.02   : 11.76227
Lift 0.01   : 20.61515
Lift 0.004  : 41.56279
Lift 0.0036 : 46.18109
Lift 0.0032 : 50.39539
Lift 0.0028 : 56.40681
Lift 0.0024 : 62.69117
Lift 0.002  : 72.73489
Lift 0.0016 : 87.80228
Lift 0.0012 : 114.99346
Lift 0.0008 : 162.09651
Lift 0.0004 : 286.80046.


In [6]:
print(metrics(y_true = data['approve_label'].fillna(0),y_score = data['score2'].fillna(-100),
              lift = [0.5,0.3,0.2,0.1,0.05,0.03,0.02,0.01] + [10.**4*i /(25 * 10**6) for i in range(10,0,-1)], return_str = True))

Sample size : 25031520.00000
Posit share : 0.00005
AUC ROC     : 0.73756
AUC PR      : 0.00457
Log loss    : 0.00166
Lift 0.5    : 1.58770
Lift 0.3    : 2.20837
Lift 0.2    : 2.62261
Lift 0.1    : 4.23940
Lift 0.05   : 5.93516
Lift 0.03   : 8.42339
Lift 0.02   : 11.22195
Lift 0.01   : 20.28264
Lift 0.004  : 40.10810
Lift 0.0036 : 42.71750
Lift 0.0032 : 45.71953
Lift 0.0028 : 51.95364
Lift 0.0024 : 59.22757
Lift 0.002  : 70.24112
Lift 0.0016 : 84.68504
Lift 0.0012 : 99.06063
Lift 0.0008 : 131.96319
Lift 0.0004 : 232.76559.


In [7]:
print(metrics(y_true = data['approve_label'].fillna(0),y_score = data['score3'].fillna(-100),
              lift = [0.5,0.3,0.2,0.1,0.05,0.03,0.02,0.01] + [10.**4*i /(25 * 10**6) for i in range(10,0,-1)], return_str = True))

Sample size : 25031520.00000
Posit share : 0.00005
AUC ROC     : 0.73031
AUC PR      : 0.00438
Log loss    : 0.00166
Lift 0.5    : 1.57440
Lift 0.3    : 2.14464
Lift 0.2    : 2.62677
Lift 0.1    : 4.13965
Lift 0.05   : 5.93516
Lift 0.03   : 8.83902
Lift 0.02   : 12.09477
Lift 0.01   : 20.19952
Lift 0.004  : 36.36744
Lift 0.0036 : 38.56121
Lift 0.0032 : 42.60229
Lift 0.0028 : 46.90671
Lift 0.0024 : 52.99309
Lift 0.002  : 60.68168
Lift 0.0016 : 75.85285
Lift 0.0012 : 96.28970
Lift 0.0008 : 136.11951
Lift 0.0004 : 222.37427.


In [71]:
print(metrics(y_true = data['full_app_label'].fillna(0),y_score = data['score3'].fillna(-100),
              lift = [0.5,0.3,0.2,0.1,0.05,0.03,0.02,0.01,0.005,0.001,0.0005,0.0001], return_str = True))

Sample size : 25031520.00000
Posit share : 0.00028
AUC ROC     : 0.75486
AUC PR      : 0.01469
Log loss    : 0.00972
Lift 0.5    : 1.61039
Lift 0.3    : 2.24934
Lift 0.2    : 2.86890
Lift 0.1    : 4.48638
Lift 0.05   : 7.24461
Lift 0.03   : 10.74064
Lift 0.02   : 14.33741
Lift 0.01   : 23.75144
Lift 0.005  : 39.95479
Lift 0.001  : 125.71203
Lift 0.0005 : 203.75812
Lift 0.0001 : 360.40781.


In [104]:
import pandas as pd
quantiles = [0.0015,0.001,0.0004,0.0002]
df_quantiles = pd.DataFrame(index = quantiles)
df_quantiles['score1_q'] = data.loc[:,'score1'].quantile(1 - np.array(quantiles)).values
df_quantiles['score2_q'] = data.loc[:,'score2'].quantile(1 - np.array(quantiles)).values
df_quantiles['score3_q'] = data.loc[:,'score3'].quantile(1 - np.array(quantiles)).values
df_quantiles['cnt'] = np.array([0.0015,0.001,0.0004,0.0002])* data.shape[0]
df_quantiles

,score1_q,score2_q,score3_q,cnt
0.0015,-7.484206,-6.366194,-5.727061,37547.280
0.0010,-7.206221,-6.229065,-5.458266,25031.520
0.0004,-6.320478,-5.741620,-5.086167,10012.608
0.0002,-5.677153,-5.121159,-4.576494,5006.304


Для тестирования увеличения охвата снижаем порог отсечения:
'la_apppr_ccall_2_1' -7.2
'la_apppr_ccall_2' -6.23
'la_stand_ccall_2э -5.46